In [19]:
import pandas as pd
import numpy as np

import operator as op
import re

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

# Custom Transformers
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector

In [20]:
df = pd.read_csv("Data/listings.csv", sep = ",")

In [21]:
class ColumnDroppersTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def transform(self, X, y=None):
        
        print("Start - ColumnDroppersTransformer")
        
        # Find features related to host, id and url
        re_drop = ".*host.*|.*id.*|.*url.*"
        drop_feat = []
        for feat in X.columns:
            if re.match(re_drop, feat):
                 drop_feat.append(feat)
        # Add extra features that for sure will not ber part of the model
        drop_feat.extend(["calendar_updated", "license", "neighbourhood_group_cleansed", "neighbourhood", "neighborhood_overview", 
                          "last_scraped", "source", "first_review", "last_review", "name", "number_of_reviews_l30d", "number_of_reviews",
                          "availability_30","availability_60","availability_90", "minimum_nights", "maximum_nights", "review_scores_value",  
                         "review_scores_accuracy", "review_scores_rating", "review_scores_checkin", "review_scores_cleanliness", "review_scores_communication",
                         "has_availability", "instant_bookable",])
        
        X = X.drop(drop_feat, axis = 1)
        
        print("End - ColumnDroppersTransformer")
        
        return X
    
    def fit(self, X, y=None):
        return self
    
class DropNasTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features
    
    def transform(self, X, y=None):
        
        print("Start - DropNasTransformer - transform")
        
        
        X = X.dropna(subset = self.features)
        
        print(type(X))
        print("End - DropNasTransformer - transform")
        
        
        return X
    
    def fit(self, X):
        
        
        return self
    
class OutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, features_limit, mode = "remove", operator = "eq"): #st: smaller then
                    
        self.opt_dict = {
        "lt":op.lt,
        "le":op.le,
        "eq":op.eq,
        "ne":op.ne,
        "ge":op.ge,
        "gt":op.gt,
        }    
        
        self.features_limit = features_limit # list of tuples
        self.mode = mode  #mode of operation (cap value or remove value)
        self.operator = operator #lt, eq, gt, ...
     
    
    def transform(self, X, y=None):
        
        print("Start - OutlierRemover")

        NAME = 0
        LIMIT = 1
        if(self.mode == "remove"): ###!!! check if it works
            for a_feature in self.features_limit:
                X = X.drop(X[self.opt_dict[self.operator](X[a_feature[NAME]], a_feature[LIMIT])].index)
        elif(self.mode == "cap"):
            for a_feature in self.features_limit:
                X[a_feature[NAME]] =  X[a_feature[NAME]].apply(lambda x : x if self.opt_dict[self.operator](x,a_feature[LIMIT]) else a_feature[LIMIT])
        
        print("End - OutlierRemover")
        
        
        return X
    
    def fit(self, X):
        return self
        
        
class ClusterGeolocationTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, clusters = 8, init = 'k-means++', n_init = 'auto', max_iter = 300):
        
        self.clusters = clusters
        self.init = init
        self.n_init = n_init
        self.max_iter = max_iter
    
    def transform(self, X, y=None):
        
        print("Start - ClusterGeolocationTransformer")
        
        # Initialize the model
        k_means = KMeans(
            init = self.init,
            n_clusters = self.clusters,
            n_init = self.n_init,
            max_iter = self.max_iter,
            random_state = 69
        )


        #Select the data to be clustered
        df_kmeans = X.loc[:,["latitude", "longitude"]]

        # Fitting
        k_means.fit(df_kmeans)

        #Clusters as a feature
        X['cluster'] = k_means.labels_ 
        
        #Drop latitude and longitude as they will not be fed into the model
        X = X.drop(["longitude", "latitude"], axis = 1)
        
        print("End - ClusterGeolocationTransformer")
        
        return X
    
    def fit(self, X):
        return self



class PreprocessCorpus(BaseEstimator, TransformerMixin):
    def __init__(self, corpus_feature):
        self.corpus_feature = corpus_feature
    
    def transform(self, X, y=None):
        
        print("Start - PreprocessCorpus")
        
        def process_corpus(corpus):
            corpus = corpus.lower()
            corpus = re.sub(r'[^a-zA-Z ]', ' ', corpus)
            corpus = re.sub(r'\b(the|and|in|with|to|a|of|br|is|from|for|on|this|you|it|has|all|at|de|by|br)\b', ' ', corpus)
            corpus = re.sub(r'br\b', '', corpus)
            return corpus
    
        X[self.corpus_feature] = X[self.corpus_feature].apply(lambda x : process_corpus(x))
        
        print("End - PreprocessCorpus")
        
        return X
    
    def fit(self, X):
        return self    


class ContainWordsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, words, new_feature_name, corpus_target):
        self.words = words
        self.feature_name = new_feature_name
        self.corpus_target = corpus_target
    
    def transform(self, X, y=None):
        
        print("Start - ContainWordsTransformer")
        
        def check_lux_corpus(x):
            
            # Transform list of words in regex
            w_list = []
            for i in self.words:
                w_list += w_list + i + "|"
            #remove last "|"
            w_list.pop()  
            regex = r'\b('+w_list+')\b'
            
            # Return if description contains lux word or not
            if (re.search(regex, x)):
                return 1
            else:
                return 0
        
        #Create the feature that identify a property as luxurious
        X[self.new_feature_name] = X[self.corpus_target].apply(check_lux_corpus)
        
        # Drop corpus target feature
        X = X.drop(self.corpus_target, axis = 1)
        
        print("End - ContainWordsTransformer")
        
        return X
    
    def fit(self, X):
        return self
    
    
class ExtractAmenitiesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, amenities_dict):
        self.amenities_dict = amenities_dict
    
    def transform(self, X, y=None):
        
        print("Start - ExtractAmenitiesTransformer")
        
        #This function will check the presence of the amenities in the list using regex
        def convert_amenities(amenities, index): 
            for amn_name, amn_re in self.amenities_dict.items():
                if(isinstance(amn_re,list)):
                    # Check each reg from the list of reg for an specific amenity. Ex ("seaview" : [".*beach view.*",".*sea view.*",".*ocean view.*"])
                    for reg in amn_re:
                        if(re.match(reg, str.lower(amenities))):
                            X.loc["index,has_"+ amn_name] = 1
                else:
                    if(re.match(amn_re, str.lower(amenities))):
                        X.loc[index,"has_"+ amn_name] = 1
        
        # 1) Create and initalize the features from the amenities list
        has_amn_feat = ["has_" + x for x in self.amenities_dict.keys()]
        X.has_amn_feat = 0
        
        # 2) For each amenities' list, check if they contain the amenities target
        for index, row in X.iterrows():
            convert_amenities(X.loc[index, "amenities"], index)
        
        # 3) Drop 'amenities' feature
        X = X.drop("amenities", axis = 1)
        
        print("End - ExtractAmenitiesTransformer")
        
        return X
    
    def fit(self, X):
        return self
    
    
   
    
class ExtractBathroom(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def transform(self, X, y=None):

        print("Start - ExtractBathroom")
        
        def bathroom_number(a_bath):
            
            is_shared = 0 #default value

            # Case if it is a missing value
            if(pd.isna(a_bath)):
                number_bath = np.nan
                is_shared = np.nan
                
            # Check if bathroom is shared using regex
            else:
                if(re.match(".*shared.*", str.lower(a_bath))):
                        is_shared = 1  


            return is_shared
            
    

        X[["is_bathroom_shared"]] = X["bathrooms_text"].apply(lambda x: pd.Series(bathroom_number(x)))
        
        X = X.drop("bathroom_text", axis = 1)
        
        print("End - ExtractBathroom")
        
        return X
    
    def fit(self, X):
        return self
    

class CatImputer(BaseEstimator, TransformerMixin):
    def __init__(self, features_limits):
        self.features_limits = features_limits
    
    def transform(self, X, y=None):
        
        print("Start - CatImputer")
        
        FILL_VALUE = 1
        FEATURE = 0
        X_imp = X
        for feat in self.features_limits:
            imp = SimpleImputer(strategy='constant', fill_value= feat[FILL_VALUE])
            X_imp = imp.fit_transform(X_imp)
        
        print("End - CatImputer")
        
        return pd.DataFrame(X_imp, columns = X.columns)
    
    def fit(self, X):
        return self  
    
    
class PriceTypeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def transform(self, X, y=None):
        
        print("Start - PriceTypeTransformer - transform")
        
        X["price"] = X["price"].apply(lambda x: float(x[1:].replace(",","")) if pd.notna(x) else x)
        
        print("End - PriceTypeTransformer - transform")
        
        return X
    
    def fit(self, X):
        return self  
    

class NumImputer(BaseEstimator, TransformerMixin):
    def __init__(self, value):
        self.value = value
    
    def transform(self, X, y=None):
        
        print("Start - FeaturesImputer - transform")
        
        impt = SimpleImputer(strategy='constant', fill_value = self.value)
        impt_X = impt.fit_transform(X)
        
        print("End - FeaturesImputer - transform")
        
        
        return pd.DataFrame(impt_X, columns = X.columns)
    
    def fit(self, X):
        
        
        return self

In [22]:
target_feature = "price"


# Separte target feature from the dataset
#X = df.drop(target_feature, axis = 1)
#y = df[target_feature]

X = df


num_feat_extraction_pipeline = Pipeline([
    ("clusterlocation", ClusterGeolocationTransformer(clusters = 25, init = "random", n_init = 15, max_iter = 1000)),
    
       
    # binary is_score_empty (?) - maybe info is reduntant
])

##################################
##################################

cat_feat_extraction_pipeline = Pipeline([
    #Removes capital letters, ponctuation, etc...
    ("preprocesstext", PreprocessCorpus("description")),
     #Creates a new feature based on lux words
    ("extractluxdescription", ContainWordsTransformer(new_feature_name = "contains_lux_description", 
                                               corpus_target = "description",
                                               words = ["lux","luxurious","luxury","fancy","garage", 
                                                  "hydromassage", "cellar", "sophistication", 
                                                  "magnificent", "colonial", "rooftop", "triplex", "suite"])), 
    
    ("extractamenities", ExtractAmenitiesTransformer(
                                                { "parking": ".*parking on premises.*",
                                                  "pool":".*pool.*(?!.*\btable\b).*",
                                                  "washer": ".*washer.*",
                                                  "dishwasher": ".*dishwasher.*",
                                                 "ceiling_fan" : ".*ceiling fan.*",
                                                 "long_term" : ".*long term.*",
                                                 "bbq_grill" : ".*bbq grill.*",
                                                 "outdoor": ".*outdoor.*",
                                                 "hot_tub": ".*hot tub.*",
                                                 "bathtub": ".*bathtub.*",
                                                 "ac": [".*air conditioning.*","\\bac\\b"],
                                                 "seaview" : [".*beach view.*",".*sea view.*",".*ocean view.*"]
                                                }
    )),
    
    ("extractbathroom", ExtractBathroom()),
    ("extractroomtype", OneHotEncoder(sparse_output = False, feature_name_combiner='concat')),
])

##################################
##################################

num_pipeline = Pipeline([
    ("dropnas", DropNasTransformer("price")),
    ("num_imputer", NumImputer(value = 0)),
    ("outliers", OutlierRemover(
                                features_limit = [("price", 1000),
                                                 ("minimum_nights_avg_ntm", 7),
                                                 ("beds", 8),
                                                 ("bedrooms", 5),
                                                 ("bathrooms", 5),
                                                 ("accommodates", 10),
                                                 ("number_of_reviews_ltm", 25),
                                                 ("reviews_per_month", 4),
                                                ],
                                mode = "cap",
                                operator = "lt" #operation: less then (lt)
                                
                                )),
    
])

##################################
##################################

cat_pipeline = Pipeline([
    ("cat_imputer", CatImputer(
                            features_limits = [("bathroom_text", "Private bath"),
                                               ("description", ""),
                                              ]
    )),
    ("outliers", OutlierRemover(
                                features_limit = [("room_type", "Hotel room")],
                                mode = "remove",
                                operator = "eq",
    )),

])

##################################
##################################


preprocessor_pipeline = Pipeline([   
    ('pricetype', PriceTypeTransformer()),
    ('preprocessor', ColumnTransformer([     
                                        ("num_ct", num_pipeline, make_column_selector(dtype_include = np.number)), #make_column_selector (to dinamically get the columns names)
                                        ("cat_ct", cat_pipeline, make_column_selector(dtype_exclude = np.number)),
                                    ])), 
    ('initialdrop', ColumnDroppersTransformer()), #Add this to feature_selection (?) (!)
])

##################################
##################################

#onehotencoder = ColumnTransformer(["onehotencoder",
#                                          OneHotEncoder(sparse_output = False, feature_name_combiner='concat'),
#                                          X.select_dtypes(include = "object")])

model_pipeline = Pipeline([
    ('preprocessor', preprocessor_pipeline),
    ("num_feature_extraction", num_feat_extraction_pipeline),
    ("cat_feat_extraction_pipeline", cat_feat_extraction_pipeline),        
    ("onehotencoder", ColumnTransformer([
                                            (OneHotEncoder(sparse_output = False, feature_name_combiner='concat'), X.select_dtypes(include = "object"))
                                        ])),

    ("normalisation", MinMaxScaler()),
    # Feature selection (remove low correaltion, low impaact, etc...)
    # Model training
    
])

##################################
##################################

In [23]:
#Training
model_pipeline.fit(X)

#Predict
#pred = model_pipeline.predict(X)

Start - PriceTypeTransformer - transform
End - PriceTypeTransformer - transform
Start - DropNasTransformer - transform
<class 'pandas.core.frame.DataFrame'>
End - DropNasTransformer - transform
Start - FeaturesImputer - transform
End - FeaturesImputer - transform
Start - OutlierRemover
End - OutlierRemover
Start - CatImputer
End - CatImputer
Start - OutlierRemover
End - OutlierRemover


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 33692 and the array at index 1 has size 34639

In [ ]:
#Custom transformers (for template purposes of how pipeline works)

#Drop columns
class ColumnDropperTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def transform(self, X, y=None):
        X = X.drop(self.columns, axis = 1)
        return X
    
    def fit(self, X):
        return self
    
#Filter
class DropNas(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def transform(self, X, y=None):
        X = X.dropna(subset = "bathrooms")
        X = X.dropna(subset = ["bedrooms", "beds"])
        X = X.dropna(subset = "price")
        
        return X
    
    def fit(self, X):
        return self

In [ ]:
drop_columns = ["host_id", "host_url", "host_name", "host_location", "host_about", "host_thumbnail_url", 
        "host_picture_url", "host_neighbourhood", "host_since", "host_listings_count", "host_total_listings_count",
        "id", "scrape_id", "listing_url", "picture_url", "minimum_minimum_nights", "maximum_minimum_nights", "minimum_maximum_nights", 
        "maximum_maximum_nights", "minimum_nights_avg_ntm", "maximum_nights_avg_ntm", "last_scraped", "source", "first_review", "last_review", "license", "neighbourhood",
        "neighborhood_overview", "neighbourhood_group_cleansed", "name", "description",
        "calendar_updated", "calculated_host_listings_count", "calculated_host_listings_count_entire_homes",
        "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms", "calendar_last_scraped", "longitude", "latitude","availability_365", "minimum_nights", 
        "maximum_nights", "availability_30","availability_60", "availability_90", "number_of_reviews_ltm", "number_of_reviews_l30d", "number_of_reviews",
               "review_scores_value", "review_scores_rating", "review_scores_accuracy",
               "review_scores_checkin", "review_scores_cleanliness", "review_scores_communication",
               "review_scores_location" ,"reviews_per_month", "has_availability", "instant_bookable", "host_response_rate", "host_acceptance_rate", "host_is_superhost", "host_has_profile_pic", "host_identity_verified",
                "host_response_time", "property_type", "host_verifications"
                
                
       ]

drop_transformer = ColumnDropperTransformer(drop_columns)

df_transformed = drop_transformer.fit_transform(df)
print(df_transformed.isna().sum())
df_transformed.columns

In [ ]:
#Option 1: Naming the transformers
my_pipeline_named = Pipeline([
    ("dropnas", DropNas()),
    ("dropcolumns", ColumnDropperTransformer(columns = drop_columns)),
])

#Option 2: NOT naming the transformers
my_pipeline_unamed = make_pipeline(DropNas(), 
                            ColumnDropperTransformer(columns = drop_columns))

In [ ]:
df_transformed = my_pipeline_named.fit_transform(df)
df_transformed.columns

In [ ]:
Implement the idea of the code bellow where we separate ccategorical and numerical data and apply5 different transofrmes to each (!)
using ColumnTransformers

from sklearn.compose import ColumnTransformer

num_attribs = ["longitude", "latitude", "housing_median_age", "total_rooms",
               "total_bedrooms", "population", "households", "median_income"]
cat_attribs = ["ocean_proximity"]

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

# ---

In [ ]:
re_drop = ".*host.*|.*id.*|.*url.*"

drop_feat = []

for feat in df.columns:
    if re.match(re_drop, feat):
         drop_feat.append(feat)
            
drop_feat.extend(["calendar_updated", "license", "neighbourhood_group_cleansed", "neighbourhood", "neighborhood_overview", "last_scraped", "source", "first_review", "last_review", "name"])

In [ ]:
df.columns